In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numba import jit
import math

import xgboost as xgb

In [16]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [17]:
train_1_reduce = train_1[0:100]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Lovelyz_zh.wikipedia.org_all-access_spider,Lulu_zh.wikipedia.org_all-access_spider,MADTOWN_zh.wikipedia.org_all-access_spider,MAMAMOO_zh.wikipedia.org_all-access_spider,MONSTA_X_zh.wikipedia.org_all-access_spider,Madame_Antoine_zh.wikipedia.org_all-access_spider,Mamamoo_zh.wikipedia.org_all-access_spider,Mike_D._Angelo_zh.wikipedia.org_all-access_spider,Miss_A_zh.wikipedia.org_all-access_spider,Missing9_zh.wikipedia.org_all-access_spider
Dates,,,,,,,,,,,,,,,,,,,,,
2015-07-01,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,3.0,0.0
2015-07-02,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,6.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,9.0,0.0
2015-07-03,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,5.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,12.0,0.0
2015-07-04,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,21.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,14.0,0.0
2015-07-05,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,4.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0


In [18]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [19]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE'])

for column in train_1_reduce.columns:
    df = pd.DataFrame(train_1_reduce[column])
    lag = 8
    for i in range(lag):
        print(i)
        df['diff' + str(i+1)] = 0
        df['diff' + str(i+1)] = df[column].diff(i+1)
    X = df.iloc[:,1:9]
    Y = df.iloc[:,0]
    Xtrain = X[0:500]
    Xtest = X[500:550]
    Ytrain = Y[0:500]
    Ytest = Y[500:550]
    
    dtrain = xgb.DMatrix(Xtrain, label=Ytrain)
    dtest = xgb.DMatrix(Xtest, label=Ytest)
    param = {'max_depth':2, 'eta':1, 'objective=reg' :' tweedie'}
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(dtest)
    dfPredictions = pd.DataFrame(Ytest)
    dfPredictions['predictions'] = preds
    rmse = np.sqrt(mean_squared_error(dfPredictions[column], dfPredictions['predictions']))
    smape = smape_fast(dfPredictions[column].values,  dfPredictions['predictions'].values)
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    metrics_df.loc[column] = [rmse,smape]

0
1
2
3
4
5
6
7
[10:55:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 7.413
Test SMAPE: 24.791
0
1
2
3
4
5
6
7
[10:55:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 65.274
Test SMAPE: 25.848
0
1
2
3
4
5
6
7
[10:55:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:5

3
4
5
6
7
[10:55:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 9.113
Test SMAPE: 31.978
0
1
2
3
4
5
6
7
[10:55:11] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 6.497
Test SMAPE: 24.563
0
1
2
3
4
5
6
7
[10:55:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Pa

4
5
6
7
[10:55:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 2.903
Test SMAPE: 29.576
0
1
2
3
4
5
6
7
[10:55:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 5.239
Test SMAPE: 35.182
0
1
2
3
4
5
6
7
[10:55:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Para

[10:55:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 8.887
Test SMAPE: 32.419
0
1
2
3
4
5
6
7
[10:55:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 8.474
Test SMAPE: 36.446
0
1
2
3
4
5
6
7
[10:55:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: 

0
1
2
3
4
5
6
7
[10:55:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 4.230
Test SMAPE: 21.650
0
1
2
3
4
5
6
7
[10:55:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 5.355
Test SMAPE: 25.902
0
1
2
3
4
5
6
7
[10:55:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:51

In [20]:
metrics_df

,RMSE,SMAPE
2NE1_zh.wikipedia.org_all-access_spider,7.413199,24.791398
2PM_zh.wikipedia.org_all-access_spider,65.274429,25.847587
3C_zh.wikipedia.org_all-access_spider,10.866750,44.757423
4minute_zh.wikipedia.org_all-access_spider,4.626954,22.701866
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,8.024017,110.193804
...,...,...
Madame_Antoine_zh.wikipedia.org_all-access_spider,4.207809,50.791637
Mamamoo_zh.wikipedia.org_all-access_spider,50.418281,74.320481
Mike_D._Angelo_zh.wikipedia.org_all-access_spider,6.626049,59.572475
Miss_A_zh.wikipedia.org_all-access_spider,4.978526,30.874532


In [21]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()

In [22]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)

Test moyenne RMSE: 11.703
Test somme RMSE: 1170.262
Test moyenne SMAPE: 44.310
